# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kapa'a,22.0752,-159.3190,25.88,84,75,7.20,US,1726551207
1,1,tergnier,49.6561,3.3011,11.69,97,59,5.18,FR,1726551208
2,2,port-aux-francais,-49.3500,70.2167,4.50,97,100,14.20,TF,1726551209
3,3,port blair,11.6667,92.7500,28.02,83,75,3.09,IN,1726551211
4,4,camacupa,-12.0167,17.4833,21.52,52,1,0.99,AO,1726551212


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

import geoviews as gv
from geoviews import tile_sources as gts

# Create a 'size' column based on humidity for visualization purposes
city_data_df['size'] = city_data_df['Humidity'] / 2  # Adjust the scaling factor if needed

# Create the map using hvplot with points based on city coordinates and humidity for point size
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    size='size', color='City', hover_cols=['City', 'Humidity', 'Max Temp', 'Wind Speed'], 
    tiles='OSM', geo=True, frame_width=800, frame_height=500, title="City Humidity Map"
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,size,Humidity,Max Temp,Wind Speed)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,size
34,34,port augusta,-32.5000,137.7667,23.50,12,0,3.56,AU,1726551249,6.0
55,55,chiredzi,-21.0500,31.6667,25.25,39,0,4.20,ZW,1726551272,19.5
195,195,siwa oasis,29.2041,25.5195,26.59,49,0,4.42,EG,1726551431,24.5
271,271,san patricio,28.0170,-97.5169,23.97,78,0,3.41,US,1726551521,39.0
291,291,tete,-16.1564,33.5867,22.05,40,0,1.03,MZ,1726551584,20.0


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to store the hotel names
hotel_df["Hotel Name"] = ""

# Display the sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
34,port augusta,AU,-32.5000,137.7667,12,
55,chiredzi,ZW,-21.0500,31.6667,39,
195,siwa oasis,EG,29.2041,25.5195,49,
271,san patricio,US,28.0170,-97.5169,78,
291,tete,MZ,-16.1564,33.5867,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
     "categories": "accommodation.hotel",
    "apiKey": geoapify_key,  #  Geoapify API Key
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port augusta - nearest hotel: Hotel Commonwealth
chiredzi - nearest hotel: Nesbitt Arms Hotel
siwa oasis - nearest hotel: Flower of the city hotel
san patricio - nearest hotel: No hotel found
tete - nearest hotel: Zambezi Hotel
ardestan - nearest hotel: Tourism Hotel
boca do acre - nearest hotel: No hotel found
alaghsas - nearest hotel: Hôtel Zagado
sao raimundo nonato - nearest hotel: Pousada Lelinha
nova vicosa - nearest hotel: Pousada Pontal da Barra
ar rutbah - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
babol - nearest hotel: هتل رضا


,City,Country,Lat,Lng,Humidity,Hotel Name
34,port augusta,AU,-32.5000,137.7667,12,Hotel Commonwealth
55,chiredzi,ZW,-21.0500,31.6667,39,Nesbitt Arms Hotel
195,siwa oasis,EG,29.2041,25.5195,49,Flower of the city hotel
271,san patricio,US,28.0170,-97.5169,78,No hotel found
291,tete,MZ,-16.1564,33.5867,40,Zambezi Hotel
296,ardestan,IR,33.3761,52.3694,26,Tourism Hotel
321,boca do acre,BR,-8.7522,-67.3978,50,No hotel found
355,alaghsas,NE,17.0187,8.0168,65,Hôtel Zagado
395,sao raimundo nonato,BR,-9.0153,-42.6994,41,Pousada Lelinha
457,nova vicosa,BR,-17.8919,-39.3719,85,Pousada Pontal da Barra


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat', 
    size=10, color='City', hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'], 
    tiles='OSM', geo=True, frame_width=800, frame_height=500, title="City and Hotel Map"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name,Humidity)